<a href="https://colab.research.google.com/github/visith1577/tensorflow_advanced/blob/main/custom_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow.keras.backend as K

In [2]:
model = tf.keras.Sequential(
    [
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(128),
     tf.keras.layers.Lambda(lambda x: tf.abs(x)),
     tf.keras.layers.Dense(10, activation= 'softmax')
    ]
)

In [4]:
def custom_relu(x):
  return K.maximum(-0.1, x)

In [5]:
model = tf.keras.models.Sequential([ 
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(128) ,
    tf.keras.layers.Lambda(custom_relu) , 
    tf.keras.layers.Dense(10, activation='softmax')

])

In [6]:
from tensorflow.keras.layers import Layer

In [7]:
class MyDense(Layer) : 
    
    def __init__(self, units = 32, activation = None) : 
        super().__init__()  
        self.units = units   
        self.activation = tf.keras.activations.get(activation) 

    def build(self, input_shape) : # create the state of the layer weights 
        
        w_init = tf.random_normal_initializer() 

        self.w = tf.Variable(name = 'kernal', 
                    initial_value= w_init(shape = (input_shape[-1], self.units), 
                       dtype= 'float32') , trainable= True)  

        b_init = tf.zeros_initializer() 

        self.b = tf.Variable(name = 'bias', 
                                initial_value= b_init(shape = (self.units,), 
                                 dtype= 'float32'), trainable= True)   
        
        super().build(input_shape) 

    def call(self, inputs) :   # defines the computation from inputs to outputs 
        return self.activation(tf.matmul(inputs, self.w) + self.b) 
        

In [8]:
my_dense = MyDense(units= 1) 

x = tf.ones((1,1)) 
y = my_dense(x) 

print(my_dense.variables) 

[<tf.Variable 'my_dense/kernal:0' shape=(1, 1) dtype=float32, numpy=array([[0.00579431]], dtype=float32)>, <tf.Variable 'my_dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


In [10]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    MyDense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2957 - accuracy: 0.9142
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1431 - accuracy: 0.9571
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1090 - accuracy: 0.9666
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0883 - accuracy: 0.9722
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 0.0752 - accuracy: 0.9765


[0.07521174103021622, 0.9764999747276306]

In [11]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
my_dense_1 (MyDense)         (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
